# Parameter-Efficient Fine-Tuning with LoRA

In this notebook, we will:
- Load a pre-trained GPT-2 model
- Evaluate its performance on a sequence classification task
- Perform parameter-efficient fine-tuning using LoRA
- Compare the performance of the fine-tuned model to the original model

In [1]:
%pip install transformers datasets peft evaluate scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Load Pre-trained Model and Tokenizer

In [2]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from datasets import load_dataset

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

/Users/lucas/Documents/repos/generative-ai/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Load Dataset

We'll use the `imdb` dataset from the Hugging Face datasets library, which is suitable for binary sequence classification.

In [3]:
# Load dataset
dataset = load_dataset("imdb")

# Preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

## Evaluate the Original Model

We'll use the Hugging Face `Trainer` to evaluate the model.

In [4]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# Define evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

# Evaluate the model
original_results = trainer.evaluate()

/Users/lucas/Documents/repos/generative-ai/venv/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/b5/17cgk42j3b51_t3dztm5slx00000gn/T/ipykernel_45829/1421716278.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Perform Parameter-Efficient Fine-Tuning with LoRA

We'll use the `peft` library to apply LoRA to the GPT-2 model.

In [5]:
from peft import LoraConfig, get_peft_model

# Create a PEFT config
peft_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1, 
    target_modules=["c_attn"],
    task_type="SEQ_CLS"
)

# Create a PEFT model
peft_model = get_peft_model(model, peft_config)

/Users/lucas/Documents/repos/generative-ai/venv/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## Train the PEFT Model

We'll fine-tune the model using the same `Trainer` setup.

In [6]:
# Update Trainer for PEFT model
trainer.model = peft_model
trainer.train_dataset=encoded_dataset["train"]
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.364900,0.338107,0.010900,0.860760
2,0.358100,0.338176,0.010900,0.863360
3,0.336600,0.323249,0.010900,0.872640


TrainOutput(global_step=9375, training_loss=0.40681127604166667, metrics={'train_runtime': 7890.4773, 'train_samples_per_second': 9.505, 'train_steps_per_second': 1.188, 'total_flos': 4916389478400000.0, 'train_loss': 0.40681127604166667, 'epoch': 3.0})

## Save the Trained Model

In [7]:
# Save the fine-tuned model
peft_model.save_pretrained("./peft_model")

## Evaluate the Fine-Tuned Model

Finally, we'll evaluate the fine-tuned model and compare its performance to the original model.

In [8]:
# Evaluate the fine-tuned model
fine_tuned_results = trainer.evaluate()

# Compare results
print("Original Model Accuracy:", original_results['eval_accuracy'])
print("Fine-Tuned Model Accuracy:", fine_tuned_results['eval_accuracy'])

Original Model Accuracy: 0.50064
Fine-Tuned Model Accuracy: 0.87264
